# **LangChain Function/Tool Calling Calculator**

A simple project that integrates LangChain's tool-calling capabilities with a conversational agent to perform basic arithmetic operations. This project demonstrates how to use LangChain's initialize_agent function to create a dynamic agent capable of interpreting and executing mathematical queries.

### **Install Dependencies**

In [2]:
!pip install -q -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.7 MB/s eta 0:00:00


### **Set Your Gemini API Key**

In [3]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

 ### **Initialize the Gemini language model using the provided API key and model name.**

In [5]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash",
    api_key=GOOGLE_API_KEY,
)

### **Test Gemini LLM**
Tests the Gemini model by asking a simple question ("What is the capital of France?").

In [6]:
llm.invoke("What is the capital of France?").content

'Paris'

### **Define the Tools**
Defines several Python functions as tools for the agent. These functions perform basic arithmetic operations like addition, subtraction, multiplication, division, power, and square root.

In [7]:
from langchain_core.tools import tool


@tool
def add(a: float, b: float) -> float:
    """Adds two numbers.

    Args:
        a: First number
        b: Second number

    Returns:
        The sum of a and b.
    """
    return a + b


@tool
def subtract(a: float, b: float) -> float:
    """Subtracts the second number from the first.

    Args:
        a: First number
        b: Second number

    Returns:
        The result of a - b.
    """
    return a - b


@tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers.

    Args:
        a: First number
        b: Second number

    Returns:
        The product of a and b.
    """
    return a * b


@tool
def divide(a: float, b: float) -> float:
    """Divides the first number by the second.

    Args:
        a: First number
        b: Second number

    Returns:
        The result of a / b.

    Note:
        If b is 0, it will return an error message.
    """
    if b == 0:
        return "Error: Division by zero is not allowed."
    return a / b


@tool
def power(base: float, exponent: float) -> float:
    """Raises a number to the power of the given exponent.

    Args:
        base: The base number
        exponent: The power to raise the base to

    Returns:
        The result of base^exponent.
    """
    return base ** exponent


@tool
def sqrt(number: float) -> float:
    """Calculates the square root of a number.

    Args:
        number: The number to find the square root of

    Returns:
        The square root of the number.
    """
    if number < 0:
        return "Error: Cannot calculate the square root of a negative number."
    return number ** 0.5


### **Gather Tools**
 Gathers the defined tools into a list to be used by the agent.

In [8]:
tools = [add, subtract, multiply, divide, power, sqrt]

### **Bind Tools and Test**

In [9]:
llm_with_tools = llm.bind_tools(tools)

In [10]:
query = "What is 3 * 12?"

llm.invoke(query)

AIMessage(content='3 * 12 = 36', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-28e7e843-4366-454b-8dd2-bb73145f57f2-0', usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}})

### **Tool Calling Workflow**
Demonstrates the tool-calling workflow using HumanMessage and ToolMessage for interaction with the LLM and tools. It processes the query, selects the appropriate tool, executes it, and appends the tool's output to the message history.


In [28]:
query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 12.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a869e78c-27dd-45da-ab70-893ede8723a9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '61d4c3b6-51c0-4434-b1ab-0aedb3e10da7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 492, 'output_tokens': 3, 'total_tokens': 495, 'input_token_details': {'cache_read': 0}})

### **Get Final Response**
 Sends the final message history to the LLM to obtain the final response, which should contain the result of the calculation

In [29]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

In [30]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

[HumanMessage(content='What is 3 * 12?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 12.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-93cf511d-0708-4f87-b037-27dafe9b5f68-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '89019b1e-56df-4c92-a53d-b0f2cc2ba30f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 492, 'output_tokens': 3, 'total_tokens': 495, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='36.0', tool_call_id='89019b1e-56df-4c92-a53d-b0f2cc2ba30f')]

In [31]:
response = llm_with_tools.invoke(messages)
response

AIMessage(content='36.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-61ff3815-46b4-4e8f-b7c7-c3010b177afa-0', usage_metadata={'input_tokens': 527, 'output_tokens': 3, 'total_tokens': 530, 'input_token_details': {'cache_read': 0}})

 ### **Displays the agent's response using Markdown for better formatting**

In [34]:
from IPython.display import Markdown

Markdown(response.content)

36.

### **Import Memory**

In [69]:
from langchain.memory import ConversationBufferMemory

 ### **Initialize the conversation memory to store the conversation history**

In [36]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

<ipython-input-36-4cbb8db76ddd>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


### **Initialize the Agent**
 Initializes the LangChain agent with the defined tools, language model, agent type, and configuration parameters.

In [67]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
# Initialize the agent using LangChain's built-in function
# Use StructuredChatZeroShotAgent instead of ZeroShotAgent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, # Changed to StructuredChatZeroShotAgent
    callback_manager=None,
    agent_kwargs={"verbose": True},  # Additional arguments like verbose mode
)

### **Start a Conversation**
Demonstrates how to interact with the initialized agent by providing a query and printing the agent's response.

In [68]:
# User query
query = "What is 3 * 12?"

# Let the agent handle the query
response = agent.run(query)

# Output the response
print("Agent's Response:", response)


<ipython-input-68-4f681d69519c>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)


Agent's Response: 36
